This code was used in Chidester et al. (2021) to calculate the shock state of olivine shock temperature samples from the flyer velocity of an Al flyer plate and the reshock Hugoniot of a quartz driver. The Al EOS is from Knudson et al. (2003), quartz EOS is from Desjarlais and Knudson (2017) (DOI: 10.1063/1.4991814) and the reshock Hugoniot is found in Fernandez-Panella (2019) (DOI: 10.1103/PhysRevLett.122.255702). The olivine Us-up relation is from other data in Chidester et al. (2021).

In [1]:
##Single sample
##Predict Us for ol

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
from scipy import linalg
from scipy.optimize import fsolve
import scipy.interpolate as interpolate
from matplotlib import rc


#Some uncertainties are assumed

def AlHug(x,C0,S1):
    return C0 + x * S1
def qtzhug_eq(x,a,b,c,d):
    return a + b * x - c * x * np.exp(-d * x)
def qtzhug_ROOTeq(x,a,b,c,d,driv_us): #Function same as above, except equal to zero
    return a + b * x - c * x * np.exp(-d * x) - driv_us
def olHug(x,a,b,c):
    return a + b * x + c * x**2

# 1. Parameter read in
# Al
Covparams = np.genfromtxt('ZAl-WtdCovMatrix.txt')
params = np.genfromtxt('ZAl-WtdHugParams.txt');

#quartz Us-up parameters
aq = 5.447
bq = 1.242
cq = 2.453
dq = 0.4336

qtzcov = [[3.028e-3, -1.490e-4, -3.715e-3, -6.275e-4],
         [-1.490e-4, 7.839e-6, 1.448e-4, 2.752e-5],
         [-3.715e-3, 1.448e-4, 1.729e-2, 1.605e-3],
         [-6.275e-4, 2.752e-5, 1.605e-3, 1.907e-4]]

#Olivine Us-up parameters
aol = 3.93403
bol = 1.63806
col = -0.01269

olcov = [[ 1.20248663e-02, -2.45357150e-03,  1.77843160e-04],
 [-2.45357150e-03,  1.44114035e-03, -7.02918682e-05],
 [ 1.77843160e-04, -7.02918682e-05,  3.86397662e-06]]

# 2. Variables read in
flyer = 17.29 # km/s, flyer velocity of the aluminum
flyer_e = 0.06 # km/s

frhoAl = 2660 #kg/m^3, density of the Al flyer
frhoAl_e = frhoAl*0.003 # uncertainty

rhoqtz = 2650 # kg/m^3, density of driver quartz
rhoqtz_e = 10 # uncertainty

rhool = 3350 # kg/m^3, density of sample olivine
rhool_e = 20 # uncertainty

steps = 10000 #as many steps as is fast, more for more accuracy.
size = 1000

up = np.linspace(0,20,size) #big enough to cover all experimental conditions

#initialize arrays
lmat = sp.linalg.cholesky(Covparams,lower=True)
lmatqtz = sp.linalg.cholesky(qtzcov,lower=True)
lmatol = sp.linalg.cholesky(olcov,lower=True)
root = np.ones(steps)
upqtz = np.ones(steps)

#quartz reference hugoniot
qtz_usref = np.ones((size,steps))
qtz_rhoref = np.ones((size,steps))
qtz_Pmc = np.ones((size,steps))

#quartz shock state
qtz_uproot = np.ones(steps)
qtz_Usroot = np.ones(steps)
qtz_rhoroot = np.ones(steps) #shock density
qtz_Proot = np.ones(steps)

#Re-shock arrays
gamma_mc = sp.zeros((size,steps))
drhoH_re=sp.zeros((size,steps))
Pmc_re=sp.zeros((size,steps))
upmc_re=sp.zeros((size,steps))

dP=sp.zeros(size)
dPe=sp.zeros(size)
drho=sp.zeros(size)
drhoe=sp.zeros(size)


dup=sp.zeros(size)
dupe=sp.zeros(size)

dP_f_re=sp.zeros(size)
dPe_f_re=sp.zeros(size)
drho_f_re=sp.zeros(size)
drho_fe_re=sp.zeros(size)
dus_f=sp.zeros(size)
dus_fe=sp.zeros(size)

Us_s = np.ones(steps)

#Monte Carlo loop to calculate initial shock state of quartz driver
i=0
for i in range(steps) :
    #perturb density
    flyerv = flyer + flyer_e * sp.randn()
    flyrho = frhoAl + frhoAl_e * sp.randn()
    qtzrho_tmp = rhoqtz + rhoqtz_e * sp.randn()
    
    #flyer
    temp_mat=sp.randn(2,2)
    bmat=np.matmul(lmat,temp_mat)
    S1 = params[0,0] + bmat[0,0]
    C0 = params[1,0] + bmat[0,1]
    
    #qtz driver
    tempqtz_mat = sp.randn(4,1)
    bmatqtz = np.matmul(lmatqtz,tempqtz_mat)
    aqtz = aq + bmatqtz[0,0]
    bqtz = bq + bmatqtz[1,0]
    cqtz = cq + bmatqtz[2,0]
    dqtz = dq + bmatqtz[3,0]
    
    #reference quartz hugoniot
    qtz_usref[:,i] = qtzhug_eq(up,aqtz,bqtz,cqtz,dqtz)
    qtz_rhoref[:,i]=qtzrho_tmp*qtz_usref[:,i]/(qtz_usref[:,i]-up)
    qtz_Pmc[:,i]=qtzrho_tmp*qtz_usref[:,i]*up    
    qtz_Pmc[:,i]=qtz_Pmc[:,i]*(10**(-3)) #to GPa
    
    #calculate qtz shock Us from Al flyer and EOS
    def bigP(up,rho0Al,C0Al,S1Al,Vi,rho0qtz,a,b,c,d):
        return rho0Al * (C0Al + S1Al * (Vi - up)) * (Vi - up) \
                - rho0qtz * (a + b * up - c * up * np.exp(-d * up)) * up    
    qtz_uproot[i] = fsolve(bigP,5,args=(flyrho,C0,S1,flyerv,qtzrho_tmp,aqtz,bqtz,cqtz,dqtz))     
    qtz_Usroot[i] = aqtz + bqtz * qtz_uproot[i] - cqtz * qtz_uproot[i] * np.exp(-dqtz * qtz_uproot[i])    
    qtz_rhoroot[i]=qtzrho_tmp*qtz_Usroot[i]/(qtz_Usroot[i]-qtz_uproot[i])
    qtz_Proot[i]=(qtzrho_tmp*qtz_Usroot[i]*qtz_uproot[i])*(10**(-3)) #GPAj
    
    #calculate quartz reshock Hugoniot
    #Needs energy change for quartz shock state - not for the mcqueen 1970
    E_shock = (1/2)*(qtz_Proot[i]*10**9)*(1/qtzrho_tmp - 1/qtz_rhoroot[i])
    E_ref = (1/2)*(qtz_Pmc[:,i]*10**9)*(1/qtzrho_tmp - 1/qtz_rhoref[:,i])
    
    #Set Gamma array
    #function
    less = np.where(qtz_usref[:, i] <= 14.69)
    more = np.where(qtz_usref[:,i] > 14.69)
    gamma_mc[less,i] = (-1.4545 + 0.1102 * qtz_usref[less,i]) + sp.randn() * 0.036 ## Desjarlais and Knudson 2017
    gamma_mc[more,i] = (0.579 * (1 - np.exp(-0.129 * (qtz_usref[more,i] - 12.81)**1.5))) + sp.randn() * 0.036
    
    Pmc_re[:,i] = ((qtz_Pmc[:,i]*10**9 + (1/2) * qtz_Proot[i]*10**9 * qtz_rhoref[:,i] * gamma_mc[:,i] * 
                           (1/qtz_rhoroot[i] - 1/qtz_rhoref[:,i]) - qtz_rhoref[:,i] * gamma_mc[:,i] * 
                           (E_ref - E_shock) ) / ( 1- ((1/2) * qtz_rhoref[:,i] * gamma_mc[:,i] *  
                                                       (1/qtz_rhoroot[i] - 1/qtz_rhoref[:,i]))))
    
    upmc_re[:,i] = ( (Pmc_re[:,i] - qtz_Proot[i]*10**9)*(1/qtz_rhoroot[i] - 1/qtz_rhoref[:,i]))**(1/2)
    upmc_re[:,i] = upmc_re[:,i] / 1000 # to km/s
            
    Pmc_re[:,i]= Pmc_re[:,i]*10**(-9) # to GPa
    
    
#Collapse the MC Data clouds outside of the "If" statements
for i in range(0,size):
    dP[i]=np.mean(qtz_Pmc[i,:]) #Initial Driver pressure Hugoniot
    dPe[i]=np.std(qtz_Pmc[i,:])
    drho[i]=np.mean(qtz_rhoref[i,:]) #Initial Driver rho Hugoniot
    drhoe[i]=np.std(qtz_rhoref[i,:])
    dus_f[i]=np.mean(qtz_usref[i,:])#Initial Driver shock velocity Hugoniot
    dus_fe[i]=np.std(qtz_usref[i,:])

    dP_f_re[i]=np.mean(Pmc_re[i,:]) #re-shock pressure Hugoniot
    dPe_f_re[i]=np.std(Pmc_re[i,:])
    dup[i]=np.mean(upmc_re[i,:]) #re-shock up Hugoniot
    dupe[i]=np.std(upmc_re[i,:])
    #drho_f_re[i]=np.mean(drhoH_re[i,:]) #re-shock Driver rho Hugoniot
    #drho_fe_re[i]=np.std(drhoH_re[i,:])

#Collapse shock state points
dUs_re = np.mean(qtz_Usroot)
dUse_re = np.std(qtz_Usroot)
dup_re = np.mean(qtz_uproot)
dupe_re = np.std(qtz_uproot)
dP_re = np.mean(qtz_Proot)
dPe_re = np.std(qtz_Proot)
drho_re = np.mean(qtz_rhoroot)
drhoe_re = np.std(qtz_rhoroot)

#Find the intersection between sample Rayleigh line and Driver Re-shock
#in an MC loop so some arrays are needed.

#Rayleigh Arrays
Pr_mc=sp.zeros((size,steps))
Pr=sp.zeros(size)
Pre=sp.zeros(size)

#Sample Shock State, which is also the same P-Up as driver at the interface.
Ps=sp.zeros(steps)
rhos=sp.zeros(steps)
ups=sp.zeros(steps)
#Grab index to start the re-shock
tmp=min(min(np.where(drho > drho_re)))


#Start Loop
for i in range(0,steps):
    #perturb the parameters
    #ol shock    
    tempol_mat = sp.randn(3,1)
    bmatol = np.matmul(lmatol,tempol_mat)
    a = aol + bmatol[0,0]
    b = bol + bmatol[1,0]
    c = col + bmatol[2,0]
    
    us_s = olHug(up,a,b,c)
    olrho = rhool + rhool_e* sp.randn()
    #Perturb re-shock Hugoniot and params
    #dP_mc2=dP_f_re + dPe_f_re*sp.randn()
    #up_re_mc=dup_re + dupe_re * sp.randn()
    #up2 = dup + dupe* sp.randn()
    #Contruct the Rayleigh line
    Pr_mc[:,i]=(olrho*us_s*up)*(10**(-3))
    #Do linear interps for Rayleigh and re-shock Hugoniot
    rayleigh = interpolate.interp1d(up[:],Pr_mc[:,i],fill_value='extrapolate')
    re_shock = interpolate.interp1d(qtz_uproot[i] - upmc_re[(tmp+10):,i],Pmc_re[(tmp+10):,i],fill_value='extrapolate')


    def difference(up):
        return rayleigh(up)-re_shock(up)
    #Quick solve for intersection up
    ups[i]=fsolve(difference, x0=10,  xtol = 0.001, maxfev=1000)
    #With the up solved, get shock pressure and density
    Us_s[i]= olHug(ups[i],a,b,c)
    rhos[i]=olrho*Us_s[i]/(Us_s[i]-ups[i])
    Ps[i]=(olrho*Us_s[i]*ups[i])*(10**(-3))
    
#Collapse needed info
for i in range(0,size):
    Pr[i]=np.median(Pr_mc[i,:]) #Rayleigh
    Pre[i]=np.std(Pr_mc[i,:])
rhos_ss=np.mean(rhos)
rhose_ss=np.std(rhos)
Ps_ss=np.mean(Ps)
Pse_ss=np.std(Ps)
ups_ss=np.mean(ups)
upse_ss=np.std(ups)
Us_ol = np.mean(Us_s)
Us_olerr = np.std(Us_s)


#Print Driver Initial Shock State
print('Initial Driver Shock State')
print('P = ',dP_re, ' +/- ', dPe_re, 'GPa')
print('Rho = ',drho_re, ' +/- ', drhoe_re, 'kg/m^3')
print('Up = ',dup_re, ' +/- ', dupe_re, 'km/s')
print('Us = ',dUs_re, ' +/- ', dUse_re, 'km/s')


#Print Sample Shock State
print('Sample Shock State')
print('P = ',Ps_ss, ' +/- ', Pse_ss, 'GPa')
print('Rho = ',rhos_ss/1000, ' +/- ', rhose_ss/1000, 'g/cm^3')
print('Up = ',ups_ss, ' +/- ', upse_ss, 'km/s')
print('Us = ',Us_ol, ' +/- ', Us_olerr, 'km/s')

Initial Driver Shock State
P =  370.75404597259677  +/-  3.1454268490867325 GPa
Rho =  5933.989938885553  +/-  22.983706156874305 kg/m^3
Up =  8.799634028358799  +/-  0.04217376601822261 km/s
Us =  15.89980405145201  +/-  0.059419003353492775 km/s
Sample Shock State
P =  444.8029016176607  +/-  4.652381176598626 GPa
Rho =  6.634637123295686  +/-  0.08517836550592758 g/cm^3
Up =  8.106543786396132  +/-  0.04679595313209076 km/s
Us =  16.378813499829878  +/-  0.15530139431928353 km/s
